In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("data/bilstein-chunks.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 20)
docs = text_splitter.split_documents(documents)

In [3]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002")
db = Chroma.from_documents(docs, embedding_function)

c:\Users\christian.braun\OneDrive - Osborne Clarke\CB\30_Code\reranking-gpt-mistral-cohere\reranking-test\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
query = "Ich habe in einer Mail einen Kunden geschrieben, dass wir ihn vernichten würden, wenn er nicht einem Vertrag zustimmt und dass er ohne uns ein Niemand sei. Wie sollte ich damit umgehen?"
docs = db.similarity_search(query)

In [5]:
docs

[Document(page_content='dadurch selber keinen Vorteil (z.B. Gegeninformation) erlange? \\n Der Austausch ist auch auf \ninformeller, ge -sellschaftlicher und rein privater Ebene kar -tellrechtlich verboten. Es kommt nicht auf \ndie Form, sondern den Inhalt an. \\n Insbesondere der Austausch von Preisen, Preissteigerungen, \nRabatten, Kunden, Kos -ten, Zuschlägen, Umsätzen, Zahlungsbe -dingungen, Kapazitäten, Mengen und \nStra-tegien ist problematisch und in jedem Fall zu vermeiden.",', metadata={'page': 10, 'source': 'data/bilstein-chunks.pdf'}),
 Document(page_content='Ge-schäftsbedingungen ab („keine Preisab -sprache“). \\n Teilen Sie mit Ihren Wettbewerbern keine \nAbsatzmärkte, Kunden oder Kundengrup -pen auf („keine Aufteilung von Märkten oder Kunden“). \nBoykottieren Sie auch nicht gemeinsam einzelne Kunden.",  \n "ueberschrift": "Hardcore -Absprachen im Kontakt mit Wettbewerbern - DONT’S"  \n  \n "abschnitt": "Hardcore -Absprachen im Kontakt mit Wettbewerbern - DOs: \\n Treffen 

# Reranking mit OpenAI GPT-4

In [6]:
from openai import OpenAI
import time
import json

start = time.time()
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_format={"type": "json_object"},
    temperature = 0,
    messages = [
        {"role": "system", "content": "You are an expert relevance ranker. Given a list of documents and a query, your job is to determine how relevant each document is for answering the query. Your output is JSON, which is a list of documents. Each document has two field, content and score. relevance_score is from 0.0 to 100.0. Higher relevance means higher score."},
        {"role": "user", "content": f"Query: {query} Docs: {docs}"}
    ] 
)

print(f"Dauerte {time.time() - start} Sekunden für das Reranking der Docs mit GPT-4.")

Dauerte 30.584593534469604 Sekunden für das Reranking der Docs mit GPT-4.


In [7]:
scores = json.loads(response.choices[0].message.content)["documents"]
sorted_data = sorted(scores, key=lambda x: x['score'], reverse=True)
print(json.dumps(sorted_data, indent=2))

[
  {
    "content": "Ge-sch\u00e4ftsbedingungen ab (\u201ekeine Preisab -sprache\u201c). \\n Teilen Sie mit Ihren Wettbewerbern keine \nAbsatzm\u00e4rkte, Kunden oder Kundengrup -pen auf (\u201ekeine Aufteilung von M\u00e4rkten oder Kunden\u201c). \nBoykottieren Sie auch nicht gemeinsam einzelne Kunden.",
    "score": 20.0
  },
  {
    "content": "dadurch selber keinen Vorteil (z.B. Gegeninformation) erlange? \\n Der Austausch ist auch auf \ninformeller, ge -sellschaftlicher und rein privater Ebene kar -tellrechtlich verboten. Es kommt nicht auf \ndie Form, sondern den Inhalt an. \\n Insbesondere der Austausch von Preisen, Preissteigerungen, \nRabatten, Kunden, Kos -ten, Zuschl\u00e4gen, Ums\u00e4tzen, Zahlungsbe -dingungen, Kapazit\u00e4ten, Mengen und \nStra-tegien ist problematisch und in jedem Fall zu vermeiden.",
    "score": 10.0
  },
  {
    "content": "eine Entschei -dung dar\u00fcber, welche Preise, Preisbestand -teile und sonstige Gesch\u00e4ftsbedingungen \nSie f\u00fcr Ihr

# Reranking mit OpenSource ColBERT

In [8]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

c:\Users\christian.braun\OneDrive - Osborne Clarke\CB\30_Code\reranking-gpt-mistral-cohere\reranking-test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch

start = time.time()
scores = []

def maxsim(query_embedding, document_embedding):
    expanded_query = query_embedding.unsqueeze(2)
    expanded_doc = document_embedding.unsqueeze(1)

    sim_matrix = torch.nn.functional.cosine_similarity(expanded_query, expanded_doc, dim=1)

    max_sim_scores, _ = torch.max(sim_matrix, dim=2)
    avg_max_sim = torch.mean(max_sim_scores, dim=1)
    return avg_max_sim

query_encoding = tokenizer(query, return_tensors='pt')
query_embedding = model(**query_encoding).last_hidden_state.mean(dim=1)

for document in docs:
    document_encoding = tokenizer(document.page_content, return_tensors='pt', truncation=True, max_length=512)
    document_embedding = model(**document_encoding).last_hidden_state

    score = maxsim(query_embedding.unsqueeze(0), document_embedding)
    scores.append({
        "score": score.item(),
        "document": document.page_content,
    })

print(f"Dauerte {time.time() - start} Sekunden für das Reranking der Docs mit ColBERT.")

Dauerte 2.361990451812744 Sekunden für das Reranking der Docs mit ColBERT.


In [10]:
sorted_data = sorted(scores, key=lambda x: x['score'], reverse=True)
print(json.dumps(sorted_data, indent=2))

[
  {
    "score": 1.0,
    "document": "dadurch selber keinen Vorteil (z.B. Gegeninformation) erlange? \\n Der Austausch ist auch auf \ninformeller, ge -sellschaftlicher und rein privater Ebene kar -tellrechtlich verboten. Es kommt nicht auf \ndie Form, sondern den Inhalt an. \\n Insbesondere der Austausch von Preisen, Preissteigerungen, \nRabatten, Kunden, Kos -ten, Zuschl\u00e4gen, Ums\u00e4tzen, Zahlungsbe -dingungen, Kapazit\u00e4ten, Mengen und \nStra-tegien ist problematisch und in jedem Fall zu vermeiden.\","
  },
  {
    "score": 1.0,
    "document": "Ge-sch\u00e4ftsbedingungen ab (\u201ekeine Preisab -sprache\u201c). \\n Teilen Sie mit Ihren Wettbewerbern keine \nAbsatzm\u00e4rkte, Kunden oder Kundengrup -pen auf (\u201ekeine Aufteilung von M\u00e4rkten oder Kunden\u201c). \nBoykottieren Sie auch nicht gemeinsam einzelne Kunden.\",  \n \"ueberschrift\": \"Hardcore -Absprachen im Kontakt mit Wettbewerbern - DONT\u2019S\"  \n  \n \"abschnitt\": \"Hardcore -Absprachen im Kontakt

# Reranking mit OpenSource Mistral

In [11]:
import json
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

start = time.time()
client = MistralClient()
response = client.chat(
    model="mistral-medium",
    messages=[
        ChatMessage(role="system", content="You are an expert relevance ranker. Given a list of documents and a query, your job is to determine how relevant each document is for answering the query. Your output is JSON, which is a list of documents. Each document has two field, content and score. relevance_score is from 0.0 to 100.0. Higher relevance means higher score."),
        ChatMessage(role="user", content=f"Query: {query} Docs: {docs}"),
    ]
)

print(f"Dauerte {time.time() - start} Sekunden für das Reranking der Docs mit mistral-medium.")

Dauerte 43.00159239768982 Sekunden für das Reranking der Docs mit mistral-medium.


In [12]:
response.choices[0].message.content

'[\n{\n"content": "dadurch selber keinen Vorteil (z.B. Gegeninformation) erlange? \\\\n Der Austausch ist auch auf informeller, gesellschaftlicher und rein privater Ebene kartellrechtlich verboten. Es kommt nicht auf die Form, sondern den Inhalt an. \\\\n Insbesondere der Austausch von Preisen, Preissteigerungen, Rabatten, Kunden, Kos -ten, Zuschlägen, Umsätzen, Zahlungsbe -dingungen, Kapazitäten, Mengen und Stra-tegien ist problematisch und in jedem Fall zu vermeiden.",\n"score": 20.0\n},\n{\n"content": "Ge-schäftsbedingungen ab („keine Preisab -sprache“). \\\\n Teilen Sie mit Ihren Wettbewerbern keine Absatzmärkte, Kunden oder Kundengrup -pen auf („keine Aufteilung von Märkten oder Kunden“). \\nBoykottieren Sie auch nicht gemeinsam einzelne Kunden.",\n"score": 50.0,\n"metadata": {"page": 10, "source": "data/bilstein-chunks.pdf"}\n},\n{\n"content": "eine Entschei -dung darüber, welche Preise, Preisbestand -teile und sonstige Geschäftsbedingungen Sie für Ihre Produkte verlangen. Reakti

In [14]:
scores = json.loads(response.choices[0].message.content)["documents"]
sorted_data = sorted(scores, key=lambda x: x['score'], reverse=True)
print(json.dumps(sorted_data, indent=2))

JSONDecodeError: Extra data: line 21 column 1 (char 1842)